In [ ]:
from transformers import AutoTokenizer
model_checkpoint = "dmis-lab/biobert-base-cased-v1.2"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


In [ ]:
def masking_sentence(token_ids, tokenizer):
    '''
    Function to mask each token in a sentence and return the masked sentence and the corresponding label
    '''
    except_tokens = [tokenizer.cls_token_id, tokenizer.sep_token_id, tokenizer.pad_token_id]
    
    mask = np.random.binomial(1, 0.1, (len(token_ids),))   # Masking each token with 15% probability
    print("mask:",  np.where(mask)[0])
    token_ids_copy = copy.deepcopy(token_ids)
    masked_sentence = pd.DataFrame()
    labels = [-100] * len(token_ids)
    for idx, token in enumerate(token_ids):
        
        if (idx in np.where(mask)[0]) and (idx not in except_tokens):
            token_ids_copy[idx] = tokenizer.mask_token_id
            labels[idx] = token
        
    masked_sentence['masked_token_id'] = [token_ids_copy]
    masked_sentence['label_id'] = [labels]
         
    return masked_sentence

In [ ]:
token_id = [101,1107,5014,117,1748,3687,26883,1320,1104,1103,172,118,6020,14715,11179,16617,5777,1107,1103,185,7897,1571,21392,1116,171,1604,1105,171,1580,8632,15416,1348,23842,117,6142,4422,3687,26883,1320,1104,1103,21996,184,5822,11478,27105,17853,113,11769,102]

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch

# Load the BioBERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.2")

# Load the BioBERT model for token classification (POS tagging)
model = AutoModelForTokenClassification.from_pretrained("dmis-lab/biobert-base-cased-v1.2")

In [ ]:
def get_pos_tags_in_biobert_vocab(text, vocab):
    # Tokenize the text using BioBERT's tokenizer
    tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(text)))
    
    # Get the BIO-style POS tags using BioBERT
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs)
    print(outputs)
    pos_tags = torch.argmax(outputs.logits, dim=2).tolist()[0]
    
    # Create a list to store POS tags and their indices
    pos_tags_with_indices = []

    for i, token in enumerate(tokens):
        # Check if the token is in the vocabulary
        if token in vocab:
            # Get the token's index in the vocabulary
            vocab_index = vocab.index(token)

            # Get the corresponding POS tag from BioBERT
            pos_tag_id = pos_tags[i]
            pos_tag = tokenizer.convert_ids_to_tokens(pos_tag_id)
            print(pos_tag)
            # Add the token, its index, and the POS tag to the list
            pos_tags_with_indices.append((token, vocab_index, pos_tag))

    return pos_tags_with_indices

# Example usage:
text = "A G-to-A transition at the first nucleotide of intron 2 of patient 1 abolished normal splicing."
biobert_vocab = ["[CLS]", "[SEP]", "example", "tags"]

pos_tags_in_biobert_vocab = get_pos_tags_in_biobert_vocab(text, biobert_vocab)
for token, index, pos_tag in pos_tags_in_biobert_vocab:
    print(f"Token: {token}, Vocabulary Index: {index}, POS Tag: {pos_tag}")
